In [1]:
import PIL
from ultralytics import YOLO
import src
from src.convert import convert_to_braille_unicode, parse_xywh_and_class

def load_model(model_path):
    """load model from path"""
    model = YOLO(model_path)
    return model

def load_image(image_path):
    """load image from path"""
    image = PIL.Image.open(image_path)
    return image

# constants
CONF = 0.15 # or other desirable confidence threshold level
MODEL_PATH = "./weights/yolov8_braille.pt"
IMAGE_PATH = "./assets/alpha-numeric.jpeg"

# receiving results from the model
image = load_image(IMAGE_PATH)
model = YOLO(MODEL_PATH)
res = model.predict(image, save=True, save_txt=True, exist_ok=True, conf=CONF)
boxes = res[0].boxes  # first image
list_boxes = parse_xywh_and_class(boxes)

result = ""
for box_line in list_boxes:
    str_left_to_right = ""
    box_classes = box_line[:, -1]
    for each_class in box_classes:
        str_left_to_right += convert_to_braille_unicode(model.names[int(each_class)])
    result += str_left_to_right + "\n"

print(result)




0: 640x640 2 001111s, 2 010100s, 2 010110s, 1 011100, 1 011110, 2 100000s, 4 100010s, 2 100100s, 4 100110s, 1 101001, 1 101010, 1 101011, 1 101100, 1 101101, 1 101110, 2 110000s, 2 110010s, 2 110100s, 2 110110s, 2 111000s, 1 111001, 2 111010s, 1 111100, 1 111110, 1435.0ms
Speed: 16.9ms preprocess, 1435.0ms inference, 13.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\detect\predict
1 label saved to runs\detect\predict\labels
e:\BaiduSyncdisk\3课外活动\科研实践 盲汉翻译\Chinese-Braille-to-Chinese-Characters\0_Final Porject\Image recognition
e:\BaiduSyncdisk\3课外活动\科研实践 盲汉翻译\Chinese-Braille-to-Chinese-Characters\0_Final Porject\Image recognition
e:\BaiduSyncdisk\3课外活动\科研实践 盲汉翻译\Chinese-Braille-to-Chinese-Characters\0_Final Porject\Image recognition
e:\BaiduSyncdisk\3课外活动\科研实践 盲汉翻译\Chinese-Braille-to-Chinese-Characters\0_Final Porject\Image recognition
e:\BaiduSyncdisk\3课外活动\科研实践 盲汉翻译\Chinese-Braille-to-Chinese-Characters\0_Final Porject\Image recognition
e:\BaiduSyncdisk\3课

In [4]:
"""
Reference
- https://docs.streamlit.io/library/api-reference/layout
- https://github.com/CodingMantras/yolov8-streamlit-detection-tracking/blob/master/app.py
- https://huggingface.co/keremberke/yolov8m-valorant-detection/tree/main
- https://docs.ultralytics.com/usage/python/
"""
import time
import PIL

import streamlit as st
import torch
from ultralyticsplus import YOLO, render_result

from convert import convert_to_braille_unicode, parse_xywh_and_class


def load_model(model_path):
    """load model from path"""
    model = YOLO(model_path)
    return model


def load_image(image_path):
    """load image from path"""
    image = PIL.Image.open(image_path)
    return image


# title
st.title("Braille Pattern Detection")

# sidebar
st.sidebar.header("Detection Config")

conf = float(st.sidebar.slider("Class Confidence", 10, 75, 15)) / 100
iou = float(st.sidebar.slider("IoU Threshold", 10, 75, 15)) / 100

model_path = "snoop2head/yolov8m-braille"

try:
    model = load_model(model_path)
    model.overrides["conf"] = conf  # NMS confidence threshold
    model.overrides["iou"] = iou  # NMS IoU threshold
    model.overrides["agnostic_nms"] = False  # NMS class-agnostic
    model.overrides["max_det"] = 1000  # maximum number of detections per image

except Exception as ex:
    print(ex)
    st.write(f"Unable to load model. Check the specified path: {model_path}")

source_img = None

source_img = st.sidebar.file_uploader(
    "Choose an image...", type=("jpg", "jpeg", "png", "bmp", "webp")
)
col1, col2 = st.columns(2)

# left column of the page body
with col1:
    if source_img is None:
        default_image_path = "./images/alpha-numeric.jpeg"
        image = load_image(default_image_path)
        st.image(
            default_image_path, caption="Example Input Image", use_column_width=True
        )
    else:
        image = load_image(source_img)
        st.image(source_img, caption="Uploaded Image", use_column_width=True)

# right column of the page body
with col2:
    with st.spinner("Wait for it..."):
        start_time = time.time()
    try:
        with torch.no_grad():
            res = model.predict(
                image, save=True, save_txt=True, exist_ok=True, conf=conf
            )
            boxes = res[0].boxes  # first image
            res_plotted = res[0].plot()[:, :, ::-1]

            list_boxes = parse_xywh_and_class(boxes)

            st.image(res_plotted, caption="Detected Image", use_column_width=True)
            IMAGE_DOWNLOAD_PATH = f"runs/detect/predict/image0.jpg"

    except Exception as ex:
        st.write("Please upload image with types of JPG, JPEG, PNG ...")


try:
    st.success(f"Done! Inference time: {time.time() - start_time:.2f} seconds")
    st.subheader("Detected Braille Patterns")
    for box_line in list_boxes:
        str_left_to_right = ""
        box_classes = box_line[:, -1]
        for each_class in box_classes:
            str_left_to_right += convert_to_braille_unicode(
                model.names[int(each_class)]
            )
        st.write(str_left_to_right)
except Exception as ex:
    st.write("Please try again with images with types of JPG, JPEG, PNG ...")

with open(IMAGE_DOWNLOAD_PATH, "rb") as fl:
    st.download_button(
        "Download object-detected image",
        data=fl,
        file_name="image0.jpg",
        mime="image/jpg",
    )


WARNING  Ultralytics settings reset to defaults. This is normal and may be due to a recent ultralytics package update, but may have overwritten previous settings. 
View and update settings with 'yolo settings' or at 'C:\Users\Lenovo\AppData\Roaming\Ultralytics\settings.yaml'
2023-11-20 14:56:23.687 
  command:

    streamlit run d:\Anaconda3\envs\braille\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
d:\Anaconda3\envs\braille\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
yolov8m.pt: 100%|██████████| 52.1M/52.1M [00:46<00:00, 1.13MB/s]
d:\Anaconda3\envs\braille\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub. Cachin

Unable to load model='snoop2head/yolov8m-braille'. As an example try model='yolov8n.pt' or model='yolov8n.yaml'


FileNotFoundError: [Errno 2] No such file or directory: './images/alpha-numeric.jpeg'